In [1]:
import tensorflow as tf

In [31]:
output_size = 19
output_size

19

In [32]:
output_size_f = tf.cast(output_size, tf.float32)
row = tf.range(output_size_f, dtype=tf.float32)
col = tf.range(output_size_f, dtype=tf.float32)

In [33]:
output_size_f

<tf.Tensor: shape=(), dtype=float32, numpy=19.0>

In [34]:
row

<tf.Tensor: shape=(19,), dtype=float32, numpy=
array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18.], dtype=float32)>

In [35]:
rows = tf.tile(row[tf.newaxis, tf.newaxis, :, tf.newaxis, tf.newaxis], [1, output_size, 1, 1, 1])
cols = tf.tile(col[tf.newaxis, :, tf.newaxis, tf.newaxis, tf.newaxis], [1, 1, output_size, 1, 1])
rows.shape

TensorShape([1, 19, 19, 1, 1])

In [36]:
offset = tf.concat([rows, cols], -1)

In [37]:

offset = tf.tile(offset, [16, 1, 1, 3, 1])

In [39]:
offset.shape

TensorShape([16, 19, 19, 3, 2])

In [29]:
a = tf.random.normal((2,3,3))
a[:,:,1:2]

<tf.Tensor: shape=(2, 3, 1), dtype=float32, numpy=
array([[[-1.1073239 ],
        [ 0.6118456 ],
        [-0.30330676]],

       [[ 0.4392243 ],
        [-1.839088  ],
        [ 1.074497  ]]], dtype=float32)>

In [40]:
pred_xyxy = tf.random.normal((16,768,4))
pred_xyxy.shape

TensorShape([16, 768, 4])

In [23]:
im_size = tf.constant([[44,64]])
im_size = tf.tile(im_size,[16,1])

In [44]:
_im_size_h = im_size[:, 0:1]
_im_size_w = im_size[:, 1:2]
_im_size_w.shape

TensorShape([16, 1])

In [45]:
_im_size = tf.concat([_im_size_w, _im_size_h], 1)
_im_size.shape

TensorShape([16, 2])

In [46]:
_im_size = tf.reshape(_im_size, (-1, 1, 2))
_im_size.shape

TensorShape([16, 1, 2])

In [47]:
_im_size = tf.tile(_im_size, [1, tf.shape(pred_xyxy)[1], 1])
_im_size

<tf.Tensor: shape=(16, 768, 2), dtype=int32, numpy=
array([[[64, 44],
        [64, 44],
        [64, 44],
        ...,
        [64, 44],
        [64, 44],
        [64, 44]],

       [[64, 44],
        [64, 44],
        [64, 44],
        ...,
        [64, 44],
        [64, 44],
        [64, 44]],

       [[64, 44],
        [64, 44],
        [64, 44],
        ...,
        [64, 44],
        [64, 44],
        [64, 44]],

       ...,

       [[64, 44],
        [64, 44],
        [64, 44],
        ...,
        [64, 44],
        [64, 44],
        [64, 44]],

       [[64, 44],
        [64, 44],
        [64, 44],
        ...,
        [64, 44],
        [64, 44],
        [64, 44]],

       [[64, 44],
        [64, 44],
        [64, 44],
        ...,
        [64, 44],
        [64, 44],
        [64, 44]]])>

In [55]:
feats = tf.random.normal((16,16,16,255))
from keras import backend as K
grid_shape = K.shape(feats)[1:3]
K.cast(grid_shape[::1], K.dtype(feats))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([16., 16.], dtype=float32)>

In [56]:
def yolo3_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    input_shape = K.cast(input_shape, K.dtype(box_xy))
    image_shape = K.cast(image_shape, K.dtype(box_xy))

    #reshape the image_shape tensor to align with boxes dimension
    image_shape = K.reshape(image_shape, [-1, 1, 1, 1, 2])

    new_shape = K.round(image_shape * K.min(input_shape/image_shape))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    # reverse offset/scale to match (w,h) order
    offset = offset[..., ::-1]
    scale = scale[..., ::-1]

    box_xy = (box_xy - offset) * scale
    box_wh *= scale

    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # x_min
        box_mins[..., 1:2],  # y_min
        box_maxes[..., 0:1],  # x_max
        box_maxes[..., 1:2]  # y_max
    ])

    # Scale boxes back to original image shape.
    image_wh = image_shape[..., ::-1]
    boxes *= K.concatenate([image_wh, image_wh])
    return boxes